# Protocol evaluation pipeline

Comparing AI generated protocols with ground truth (gt) protocols

In [ ]:
from __future__ import annotations

# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import configparser
import logging
import os
import sys
from pathlib import Path

# Type checking imports
from typing import TYPE_CHECKING, Any, NoReturn
from collections.abc import Callable

import numpy as np
import pandas as pd
from IPython.display import Markdown
from vertexai.generative_models import Part

path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import evaluation
import video_to_protocol

config = configparser.ConfigParser()
config.read("../secrets.ini")
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [ ]:
import configparser

import vertexai

config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(project=PROJECT_ID, location="us-central1")  # europe-west9 is Paris

In [ ]:
from google.cloud import storage

os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

In [ ]:
def generate_protocols_evaluation(
    protocol_gt: list[str],
    protocol_ai: list[str],
    model_name: str = "gemini-2.5-pro-preview-03-25",
    temperature: float = 0.9,
) -> tuple[str, Any]:
    """Generate an evaluation of AI-generated protocols against ground truth protocols.

    Parameters
    ----------
    protocol_gt : list[str]
        The ground truth protocols (benchmark) represented as a list of strings
    protocol_ai : list[str]
        The AI-generated protocols to evaluate represented as a list of strings
    model_name : str, optional
        The model to use for evaluation, by default "gemini-2.5-pro-preview-03-25"
    temperature : float, optional
        Temperature setting for content generation, by default 0.9

    Returns
    -------
    tuple[str, Any]
        A tuple containing (evaluation_text, usage_metadata)

    """
    inputs = [
        """
        # Instruction

        You are an expert evaluator. Your task is to evaluate the quality of a protocol generated by an AI model by comparing it to a ground truth protocol. You will conduct a systematic, section-by-section analysis to determine how well the AI-generated protocol aligns with the ground truth protocol. You should first read the protocols carefully and then evaluate the quality of the AI-generated protocol based on the evaluation criteria below.\n
        # Evaluation

        ## Evaluation Criteria
        * Completeness: What is present in both protocols. What is present in the ground truth but missing from the AI-generated protocol. What is present in the AI-generated protocol but not in the ground truth
        * Technical Accuracy: The AI-generated protocol demonstrates scientific understanding by properly distinguishing between different techniques and equipment and using appropriate scientific terminology.
        * Logical Flow: The workflow maintains the chronological sequence of the procedure.
        * Safety: The appropriate identification and emphasis of critical cautions, warnings, and safety measures.
        * Formatting: The AI-generated protocol matches the formatting of the ground truth protocol.

        ## Rating Rubric

        For each criterion, rate the AI-generated protocol on a scale of 1-5:

        5: (Very good). The AI-generated protocol demonstrates exceptional quality in this aspect, with no significant flaws or omissions. Fully meets or exceeds the ground truth protocol.
        4: (Good). The AI-generated protocol demonstrates strong quality in this aspect, with only minor shortcomings that don't significantly impact effectiveness. Closely aligns with the ground truth protocol.
        3: (Ok). The AI-generated protocol contains most essential elements but has noticeable differences from the ground truth protocol that might slightly impact its effectiveness.
        2: (Bad). The AI-generated protocol has significant deficiencies in multiple criteria from the ground truth, missing or wrongly displaying important information that would likely impact its effectiveness.
        1: (Very bad). The AI-generated protocol fails to meet minimum standards in this aspect, with fundamental flaws or critical omissions that render the content potentially unusable or unsafe.

        ## Evaluation Steps

        * Step 1: Read the 'Ground truth protocol' thoroughly and write it down again word-by-word (Verbatim).

        * Step 2: Read the 'AI-generated protocol' thoroughly and write it down again word-by-word (Verbatim).

        * Step 3: Compare each section of the AI-generated protocol with its counterpart in the ground truth protocol:

        1. Title
        2. Abstract
        3. Materials
        5. Each step from Expected Results
        6. Figures
        7. References

        For each section, note how they fullfill the evaluation criteria (Completeness, Technical Accuracy, Logical Flow, Safety, Formatting) using the 1-5 scale based on the Rating Rubric. Treat the ground truth as the gold standard and more trustworthy protocol.

        * Step 4: Comparative Analysis Table
        Create a table summarizing your findings:

        | Section | Ground Truth Protocol | AI-Generated Protocol | Completeness Rating (1-5) | Completeness Explanation | Technical Accuracy Rating (1-5) | Technical Accuracy Explanation | Logical Flow Rating (1-5) | Logical Flow Explanation | Safety Rating (1-5) | Safety Explanation | Formatting Rating (1-5) | Formatting Explanation | Notes |
        |-|-|-|-|-|-|-|-|-|-|-|-|-|-|
        | Title | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |Misaligned/Not applicable] | [Explanation] |
        | Abstract | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | Materials - e.g. Equipment | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | Materials - e.g. Reagents | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | Procedure - Step 1 | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | Procedure - Step 2 | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | Procedure - Step 3 | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | Procedure - Step 4 | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | Procedure - Step 5 | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | [Continue for all steps] | | | | | | | | | | | | | |
        | Expected Results | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | Figures | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |
        | References | [Text from ground truth] | [Text from AI protocol] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [1-5] | [Explanation] | [Additional observations or comments] |

        * Step 5: Overall Summary:
        Overall compare all sections in total and individually for Completeness, Technical Accuracy, Logical Flow, Safety, Formatting

        * Step 6: Overall Rating
        Provide an overall rating (1-5) based on step 5 and the Rating Rubric.

        # Protocols for Evaluation

        ## Ground Truth Protocol:
        """
    ]

    inputs.extend(protocol_gt)

    inputs.extend(["## AI-Generated Protocol:"])
    inputs.extend([protocol_ai])
    inputs.extend(["# Evaluation result:"])

    evaluation, usage_metadata = video_to_protocol.generate_content_from_model(
        inputs,
        model_name=model_name,
        temperature=temperature,
    )

    return evaluation, usage_metadata


def calculate_protocol_ratings(
    df_eval: pd.DataFrame,
    completeness_col: str = "Completeness Rating (1-5)",
    accuracy_col: str = "Technical Accuracy Rating (1-5)",
    logic_col: str = "Logical Flow Rating (1-5)",
    safety_col: str = "Safety Rating (1-5)",
    formatting_col: str = "Formatting Rating (1-5)",
) -> dict:
    """Calculate average ratings across evaluation criteria from a dataframe of protocol evaluations.

    Parameters
    ----------
    df_eval : pd.DataFrame
        DataFrame containing evaluation ratings for protocols
    completeness_col : str, optional
        Column name for completeness ratings, by default "Completeness Rating (1-5)"
    accuracy_col : str, optional
        Column name for technical accuracy ratings, by default "Technical Accuracy Rating (1-5)"
    logic_col : str, optional
        Column name for logical flow ratings, by default "Logical Flow Rating (1-5)"
    safety_col : str, optional
        Column name for safety ratings, by default "Safety Rating (1-5)"
    formatting_col : str, optional
        Column name for formatting ratings, by default "Formatting Rating (1-5)"

    Returns
    -------
    dict
        Dictionary containing average ratings for each criterion and an overall rating

    """
    mean_completeness = pd.to_numeric(df_eval[completeness_col], errors="coerce").mean()
    mean_accuracy = pd.to_numeric(df_eval[accuracy_col], errors="coerce").mean()
    mean_logic = pd.to_numeric(df_eval[logic_col], errors="coerce").mean()
    mean_safety = pd.to_numeric(df_eval[safety_col], errors="coerce").mean()
    mean_formatting = pd.to_numeric(df_eval[formatting_col], errors="coerce").mean()

    return {
        "Completeness": mean_completeness,
        "Technical Accuracy": mean_accuracy,
        "Logical Flow": mean_logic,
        "Safety": mean_safety,
        "Formatting": mean_formatting,
        "Overall": np.mean(
            [mean_completeness, mean_accuracy, mean_logic, mean_safety, mean_formatting]
        ),
    }

In [ ]:
protocol_ai = """
# Connecting an IonOpticks Column to a timsTOF SCP Instrument

## Abstract
This protocol describes the procedure for connecting an IonOpticks column to a timsTOF SCP instrument. It includes steps for ensuring proper connection of the sample line, grounding the column, and setting the column oven temperature. The protocol also covers checking the instrument status and idle flow using the timsControl and HyStar software.

## Materials
### Equipment
* timsTOF SCP instrument
* IonOpticks column
* Sample line with nano connector
* Nano connector adapter
* Pliers
* Column oven
* Computer with timsControl software
* Computer with HyStar software

### Reagents
* Solvent for the IonOpticks column (as recommended by the manufacturer)

## Procedure

**Estimated timing: 5 minutes**

1. Check the software. Ensure that the timsTOF SCP instrument is in standby mode in the timsControl software. If the instrument is in operate mode, switch it to standby mode.
2. Connect the column to the sample line. a. Insert the IonOpticks column into the Ultrasource. b. Connect the nano connector adapter to the sample line. c. Connect the sample line to the column using the adapter. d. Check for leaks by visually inspecting the connection for any liquid droplets. If there are leaks, tighten the connection or replace the column or emitter.
3. Secure the connection. Use pliers to tighten the sample line connector to the column. Ensure that the connection is finger tight but not over tightened.
4. Remove the adapter. Carefully remove the nano connector adapter from the sample line.
5. Position the column oven. a. Loosen the screw on the column oven to adjust its position. b. Move the oven as close as possible to the source. c. Tighten the screw to secure the oven in place.
6. Ground the column. a. Locate the grounding screw on the column oven. b. Use the screw to ground the column to the oven. c. If the column is longer, use the additional grounding screw on the oven to ensure proper grounding.
7. Close the column oven. Ensure that the lid is securely closed.
8. Set the column oven temperature. a. Check the temperature indicator on the column oven. b. Adjust the temperature to 50°C for IonOpticks columns. The temperature is indicated by three illuminated LEDs on the column toaster. c. Wait for the temperature to stabilize. The lights will stop blinking when the temperature is stable.
9. Switch the instrument to operate mode. In the timsControl software, click the on/off button to switch the instrument to operate mode.
10. Check the idle flow. In the HyStar software, check that the idle flow is on. If it is not, right-click on the Evosep logo, select 'Preparation', then select 'Idle flow' and 'Run'.
11. Confirm signal. Verify that there is signal in the timsControl and HyStar software.

## Expected Results
After completing these steps, the IonOpticks column should be properly connected to the timsTOF SCP instrument. The instrument should be in operate mode, the idle flow should be on, and there should be signal in both the timsControl and HyStar software.

## Figures
* Figure 1: Overview of the timsTOF SCP instrument and its components.
* Figure 2: Close-up view of the column oven and the grounding screw.
* Figure 3: Screenshot of the timsControl software showing the instrument status and parameters.
* Figure 4: Screenshot of the HyStar software showing the idle flow status.

## References
* IonOpticks column manufacturer's instructions
* timsTOF SCP instrument user manual
* timsControl software user manual
* HyStar software user manual
"""

In [ ]:
path = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/ConnectingColumnSampleLine_protocolCorrect.md"

protocol_gt_uri = video_to_protocol.upload_file_to_gcs(path, bucket)
protocol_gt = [Part.from_uri(protocol_gt_uri, mime_type="text/md")]

evaluation_response, usage_metadata = generate_protocols_evaluation(
    protocol_gt,
    protocol_ai,
    model_name="gemini-2.5-pro-preview-03-25",
    temperature=0.9,
)
display(Markdown(evaluation_response))

df_eval = eval.extract_table_to_dataframe(
    evaluation,
    "Comparative Analysis",
    model_name="gemini-2.5-pro-preview-03-25",
    temperature=0.9,
)

ratings_dict = calculate_protocol_ratings(df_eval)
display(ratings_dict)

In [ ]:
if TYPE_CHECKING:
    from google.cloud.storage import Bucket

from collections import defaultdict

MIME_TYPES = {
    ".pdf": "application/pdf",
    ".jpg": "image/jpeg",
    ".jpeg": "image/jpeg",
    ".png": "image/png",
}


def prepare_all_inputs(
    protocol_video_path: str,
    protocol_path: str,
    bucket: str,
    prefix: str = "generate_protocol_video",
) -> dict:
    """Prepare all inputs for the generative model.

    This function uploads files and formats them as inputs for a generative model.

    Parameters
    ----------
    protocol_video_path : str
        Path to the file that shows the correct execution (ground truth) of the protocol.
    protocol_path : str
        Path to the protocol markdown file.
    bucket : str
        GCS bucket name for uploading the files.
    prefix : str, default="compare_protocol_video"
        Prefix for the files in GCS bucket.

    Returns
    -------
    dict
        A dictionary containing the four formatted inputs:
        'protocol_video_input', 'protocol_input'

    """
    video_uri = video_to_protocol.upload_file_to_gcs(
        protocol_video_path, bucket, prefix
    )
    file_extension = Path(video_uri).suffix.lower()[1:]
    protocol_video_input = [
        Part.from_uri(video_uri, mime_type=f"video/{file_extension}")
    ]

    uri = video_to_protocol.upload_file_to_gcs(protocol_path, bucket, prefix)
    protocol_ground_truth = [Part.from_uri(uri, mime_type="text/md")]

    return {
        "protocol_video_input": protocol_video_input,
        "protocol_input": protocol_ground_truth,
    }


def process_benchmark_dataset(
    csv_path: str | Path,
    protocol_videos_base: str | Path,
    markdown_base: str | Path,
    bucket: Bucket,
    prefix: str,
) -> dict[str, Any]:
    """Process the first two rows in the benchmark dataset CSV and prepare model inputs.

    Parameters
    ----------
    csv_path : str
        Path to the CSV file containing benchmark dataset information
    protocol_videos_base : str
        Base path to the protocol videos directory
    lab_notes_videos_base : str
        Base path to the lab notes videos directory
    markdown_base : str
        Base path to the markdown files directory
    bucket : object
        The bucket object used in the prepare_all_inputs function
    prefix : str
        Prefix for the files in GCS bucket.

    Returns
    -------
    dict
        Dictionary containing all model inputs for the first two rows in the CSV,
        with experiment names as keys

    """
    benchmark_df = pd.read_csv(csv_path, sep=";")
    unique_benchmark_df = benchmark_df[["protocol video", "protocol"]].drop_duplicates()

    all_model_inputs = {}

    for (
        _index,
        row,
    ) in (
        unique_benchmark_df.iterrows()
    ):  # for testing .head(2).iterrows() or .iloc[[13, 14]] .iloc[::2]
        lab_video_path = Path(protocol_videos_base) / row["protocol video"]
        protocol_path = Path(markdown_base) / row["protocol"]

        dict_model_inputs = prepare_all_inputs(
            lab_video_path,
            protocol_path,
            bucket,
            prefix,
        )

        experiment_name = row["protocol"].split(".")[0]
        all_model_inputs[experiment_name] = dict_model_inputs

        print(f"Processed {experiment_name}")

    return all_model_inputs


def prepare_knowledge(
    folder_path: str, bucket: str, subfolder_in_bucket: str
) -> list[Part]:
    """Prepare knowledge base by collecting and processing files from specified location.

    Parameters
    ----------
    folder_path : str
        Local folder path containing knowledge files
    bucket : str
        Cloud storage bucket name
    subfolder_in_bucket : str
        Subfolder path within the bucket

    Returns
    -------
    list[Part]
        list of Google cloud Part objects created from supported knowledge files

    """
    knowledge_uris = video_to_protocol.collect_knowledge_uris(
        folder_path, bucket, subfolder_in_bucket
    )
    knowledge = []
    file_counts = defaultdict(int)

    for file_path in knowledge_uris:
        path_obj = Path(file_path)
        file_ext = path_obj.suffix.lower()

        if file_ext in MIME_TYPES:
            mime_type = MIME_TYPES[file_ext]
            try:
                knowledge.append(Part.from_uri(file_path, mime_type=mime_type))
                file_counts[file_ext] += 1
            except (OSError, ValueError):
                logger.exception(f"Error creating Part from {file_path}")
        else:
            logger.warning(f"Unsupported file extension: {file_ext}")

    logger.info(f"Total files processed: {len(knowledge)}")
    return knowledge

In [ ]:
def evaluation_of_video_to_protocol_generation(
    protocol_gt: str, protocol_ai: str, model_name: str, temperature: float
) -> tuple[str, dict[str, Any], dict[str, Any]]:
    """Function compares an AI-generated protocol with a ground truth protocol
    and provides detailed evaluation metrics and ratings.

    Parameters
    ----------
    protocol_gt : str
        Ground truth protocol used as the reference standard
    protocol_ai : str
        AI-generated protocol to be evaluated
    model_name : str
        Name of the language model to use for evaluation
    temperature : float
        Temperature parameter for controlling randomness in model output

    Returns
    -------
    tuple[str, dict[str, Any], dict[str, Any]]
        A tuple containing:
        - evaluation (str): Detailed evaluation text with comparison analysis
        - usage_metadata (dict): Model usage metadata from evaluation generation
        - ratings_dict (dict): Numerical ratings for different evaluation criteria

    """
    
    evaluation, usage_metadata = generate_protocols_evaluation(
        protocol_gt,
        protocol_ai,
        model_name,
        temperature,
    )

    df_eval = eval.extract_table_to_dataframe(
        evaluation, "Comparative Analysis", model_name, temperature
    )

    ratings_dict = calculate_protocol_ratings(df_eval)

    return evaluation, usage_metadata, ratings_dict


def generate_protocol_and_evaluate(
    video: str,
    video_example_1: str,
    protocol_example_1: str,
    video_example_2: str,
    protocol_example_2: str,
    knowledge: str,
    model_name: str,
    temperature: float,
    selected_function: Callable[..., tuple[str, dict[str, Any]]],
) -> tuple[str, dict[str, Any], dict[str, Any], dict[str, Any]]:
    """Generate protocol from video using selected function and evaluate the result.

    Parameters
    ----------
    video : str
        Input video content to generate protocol for
    video_example_1 : str
        First example video for few-shot learning
    protocol_example_1 : str
        Protocol corresponding to the first example video
    video_example_2 : str
        Second example video for few-shot learning
    protocol_example_2 : str
        Protocol corresponding to the second example video
    knowledge : str
        Domain-specific knowledge to incorporate in protocol generation
    model_name : str
        Name of the language model to use for generation and evaluation
    temperature : float
        Temperature parameter for controlling randomness in model output
    selected_function : Callable[..., tuple[str, dict[str, Any]]]
        Protocol generation function that takes video inputs and returns
        (generated_protocol, usage_metadata)

    Returns
    -------
    tuple[str, dict[str, Any], dict[str, Any], dict[str, Any]]
        A tuple containing:
        - evaluation (str): Detailed evaluation text comparing generated vs ground truth
        - ratings_dict (dict): Numerical ratings for different evaluation criteria
        - usage_metadata_protocol_ai (dict): Model usage metadata from protocol generation
        - usage_metadata_evaluation (dict): Model usage metadata from evaluation step

    """
    protocol_ai, usage_metadata_protocol_ai = selected_function(
        video,
        video_example_1,
        protocol_example_1,
        video_example_2,
        protocol_example_2,
        knowledge,
        model_name,
        temperature,
    )
    print("# Generated protocol\n", protocol_ai)

    evaluation, usage_metadata_evaluation, ratings_dict = (
        evaluation_of_video_to_protocol_generation(
            protocol_gt, protocol_ai, model_name, temperature
        )
    )

    return (
        evaluation,
        ratings_dict,
        usage_metadata_protocol_ai,
        usage_metadata_evaluation,
    )


def protocol_generation_without_added_knowledge(
    video: str,
    video_example_1: str,
    protocol_example_1: str,
    video_example_2: str,
    protocol_example_2: str,
    knowledge: str,
    model_name: str,
    temperature: float,
) -> tuple[str, dict[str, Any]]:
    """Generate protocol using a simple prompt.

    Parameters
    ----------
    video : str
        Input video to generate protocol for
    video_example_1 : str
        First example video
    protocol_example_1 : str
        Protocol for first example video
    video_example_2 : str
        Second example video
    protocol_example_2 : str
        Protocol for second example video
    knowledge : str
        Domain knowledge to incorporate
    model_name : str
        Name of the model to use
    temperature : float
        Temperature setting for generation

    Returns
    -------
    tuple[str, dict[str, Any]]
        A tuple containing (protocol_ai, usage_metadata)

    """
    inputs = [
        """
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.
    """
    ]
    inputs.extend(
        [
            """
    # Instruction

    You work with following input:
    - Video: An instructional video that demonstrates how a researcher carries out a laboratory procedure.\n
    Your task is to analyze the provided video and to convert it into a Nature-style protocol. The goal is a clear, concise, unambiguous protocol reproducible by someone with no prior knowledge.\n
    Video:
    """
        ]
    )
    inputs.extend(video)
    inputs.extend(
        [
            """
        Protocol:
    """
        ]
    )

    protocol_ai, usage_metadata = video_to_protocol.generate_content_from_model(
        inputs,
        model_name,
        temperature,
    )

    return protocol_ai, usage_metadata


def protocol_generation_with_examples(
    video: str,
    video_example_1: str,
    protocol_example_1: str,
    video_example_2: str,
    protocol_example_2: str,
    knowledge: str,
    model_name: str,
    temperature: float,
) -> tuple[str, dict[str, Any]]:
    """Generate protocol using examples.

    Parameters
    ----------
    video : str
        Input video to generate protocol for
    video_example_1 : str
        First example video
    protocol_example_1 : str
        Protocol for first example video
    video_example_2 : str
        Second example video
    protocol_example_2 : str
        Protocol for second example video
    knowledge : str
        Domain knowledge to incorporate
    model_name : str
        Name of the model to use
    temperature : float
        Temperature setting for generation

    Returns
    -------
    tuple[str, dict[str, Any]]
        A tuple containing (protocol_ai, usage_metadata)

    """
    inputs = [
        """
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.
    """
    ]
    inputs.extend(
        [
            """
    # Instruction

    You work with following input:
    - Video: An instructional video that demonstrates how a researcher carries out a laboratory procedure.\n
    Your task is to analyze the provided video and to convert it into a Nature-style protocol. The goal is a clear, concise, unambiguous protocol reproducible by someone with no prior knowledge.
    """
        ]
    )
    inputs.extend(
        [
            """
        # ====== EXAMPLE (FOR REFERENCE ONLY) ======\n
        The following set of videos and resulting protocols should solely serve as an example and are not part of the task.\n
        Example Video 1:
        """
        ]
    )
    inputs.extend(video_example_1)
    inputs.extend(
        [
            """Example documentation 1:
    1. Describe what you can hear with timestamps:
    - No audible speech.\n
    2. Describe what you can see with timestamps:
    - 0:00-0:05 The camera pans around the back of a timsTOF mass spectrometer in a lab. A bench with various equipment is visible.
    - 0:06-0:26 A researcher takes an Eppendorf pipette, then setting volume to 1 uL.
    - 0:27-0:30 The researcher indicates that the color on the top of a pipette and of pipette tip box have to agree.
    - 0:31-0:35 the researcher attaches a pipette tip to the pipette.
    - 0:36-0:50 The researcher opens an eppendorf vial and aspirates the content.
    - 0:51-0:54 The camera moves to show the lab and the evosep instrument connected to the mass spectrometer.
    - 0:55-1:03 The researcher disconnects a tubing on the ultraSource.
    - 1:03-1:07 The researcher pipettes liquid into the fitting connecting the UltraSource of a mass spectrometer.
    - 1:07-1:08 The researcher closes conencts the tubing and fitting again.
    - 1:09-1:11 The camera focused on evosep and mass spectrometer connection.
    - 1:11-1:14 The researcher is removing the pipette tip into a wast container.
    - 1:15-1:17 The camera move far away from the evosep and the mass spectrometer.

    3. Describe the used equipment:
    - timsTOF mass spectrometer: Dark blue.
    - Evosep One LC system: Orange and gray.
    - Eppendorf pipette: White.
    - Pipette tips: Transparent with dark grey top.
    - Eppendorf tubes: Small clear plastic vials.
    - Various bottles and consumables on the lab bench.
    """
        ]
    )
    inputs.extend(["Example Protocol 1:"])
    inputs.extend(protocol_example_1)
    inputs.extend(["Example Video 2:"])
    inputs.extend(video_example_2)
    inputs.extend(
        [
            """Example documentation 2:
    1. Describe what you can hear with timestamps:
    - 0:00 - 0:08: "Hello, here I will show you how to reset the TIMS Control perspective. At the moment, it's not in the default state. This can be seen at this image which stands out."
    - 0:09 - 0:14: "For this, we click here at the middle and say reset perspective."
    - 0:15 - 0:16: "Yes, we want this."
    - 0:17 - 0:21: "Now we're back to the default view."\n
    2. Describe what you can see with timestamps:
    - 0:00 - 0:05: The screen displays the Bruker timsControl software interface. The window title indicates "timsTOFscp". Several panels are visible: "TIMS View" at the top left showing a 2D plot (likely ion mobility vs. m/z), a "Chromatogram View" prominently on the right side displaying two traces (green and red), and various instrument status indicators on the left (Automation, HyStar, Calibration, Vacuum).
    - 0:05 - 0:08: The narrator refers to the current layout as non-default, implying the position of the "Chromatogram View" is non-standard and an customized arrangement.
    - 0:08 - 0:11: The mouse cursor moves to the top-right of the software window, towards a set of icons. It hovers over an icon that looks like two overlapping rectangles, typically used for managing window layouts or perspectives.
    - 0:11 - 0:12: The mouse clicks on the perspective management icon. A dropdown menu appears with options including "Show View...", "Save Perspective As...", "Reset Perspective...", "Home", "Method Editor", "Maintenance", and "Monitoring".
    - 0:12 - 0:14: The mouse cursor moves down the dropdown menu and selects "Reset Perspective...".
    - 0:14 - 0:15: A confirmation dialog box titled "Reset Perspective" pops up, asking: "Do you want to reset the current 'Home' perspective to its defaults?". "Yes" and "No" buttons are presented.
    - 0:15 - 0:16: The mouse cursor clicks the "Yes" button in the confirmation dialog.
    - 0:17 - 0:21: The software interface panels dynamically rearrange. The "TIMS View" remains at the top. A "Spectrum View" now appears in the middle section, and the "Chromatogram View" is repositioned to the bottom of the main display area. This is indicative of the default layout for this software.
    - 0:21 - 0:23: The software is shown in its new, reset (default) perspective. The mouse cursor makes a brief circular movement over the "Source" and "Syringe Pump" control area in the lower part of the screen before the video ends.\n
    3. Describe the used equipment:
    - Bruker timsControl Software: The primary focus of the video. The window title "timsTOFscp - ... - timsControl" indicates this specific software, used for controlling Bruker TIMS-TOF series mass spectrometers.
    - Computer System: The software is running on a computer, evidenced by the VNC Viewer window frame and the Windows taskbar visible at the very bottom of the screen.
    - (Implied) Bruker timsTOF Mass Spectrometer: The software is designed to control a Trapped Ion Mobility Spectrometry Time-Of-Flight mass spectrometer (e.g., timsTOF Pro, timsTOF SCP, timsTOF fleX). The "TIMS View" and other parameters are specific to such instrumentation.
    """
        ]
    )
    inputs.extend(["Example Protocol:"])
    inputs.extend(protocol_example_2)
    inputs.extend(
        [
            """
        # ====== Beginn of Analysis Task ======\n
        ## Important: The analysis must be performed on the following video \n
        Video:
        """
        ]
    )
    inputs.extend(video)
    inputs.extend(
        [
            """
        Protocol:
    """
        ]
    )

    protocol_ai, usage_metadata = video_to_protocol.generate_content_from_model(
        inputs,
        model_name,
        temperature,
    )

    return protocol_ai, usage_metadata


def protocol_generation_with_thinking_step_by_step(
    video: str,
    video_example_1: str,
    protocol_example_1: str,
    video_example_2: str,
    protocol_example_2: str,
    knowledge: str,
    model_name: str,
    temperature: float,
) -> tuple[str, dict[str, Any]]:
    """Generate protocol using step-by-step thinking.

    Parameters
    ----------
    video : str
        Input video to generate protocol for
    video_example_1 : str
        First example video
    protocol_example_1 : str
        Protocol for first example video
    video_example_2 : str
        Second example video
    protocol_example_2 : str
        Protocol for second example video
    knowledge : str
        Domain knowledge to incorporate
    model_name : str
        Name of the model to use
    temperature : float
        Temperature setting for generation

    Returns
    -------
    tuple[str, dict[str, Any]]
        A tuple containing (protocol_ai, usage_metadata)

    """
    inputs = [
        """
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.
    """
    ]
    inputs.extend(
        [
            """
    # Instruction

    You work with following input:
    - Video: An instructional video that demonstrates how a researcher carries out a laboratory procedure.\n
    Your task is to analyze the provided video and to convert it into a Nature-style protocol. The goal is a clear, concise, unambiguous protocol reproducible by someone with no prior knowledge.\n
    ## Follow this structured approach:
    * Step 1: Go through the 'Video' completely from beginning to end.
    * Step 2: Document all observations:
        - write down what you can hear with timestamps
        - write down all actions you can see with timestamps
        - note down the equipment you can identify
    * Step 3: Convert your observations into a Nature-style protocol:
        ###Protocol Writing Guidelines
        ####Format Requirements
        1. Title (format: **# Title**)
        2. Abstract (format: **## Abstract** followed by a paragraph)
        3. Materials section with Equipment and Reagents subsections
            (format: **## Materials**
                    **### Equipment**
                    - **Item 1**
                    - **Item 2**)
        4. Procedure with estimated timing
            (format: **## Procedure**
                    *Estimated timing: X minutes*
                    1. Step one
                    2. Step two)
        5. Expected Results section (format: **## Expected Results**)
        6. Figures section (format: **## Figures**)
        7. References section (format: **## References**)

        ###Key Content Adjustments
        #### Abstract
        - Focus on the core procedure, not extensive background

        #### Procedure Section
        - Steps:
            - Focus on essential actions only.
            - Be brief in your description.
            - Give every step its own number.
            - Use subheadings to group steps such as
                '### Switch timsTOF to standby
                1. In ...
                2. Verified ...
                3. In ...
                ...'
        - Language: Use direct, action-oriented language with commonly used vocabularies
        - Estimated timing: Use the video legth
    """
        ]
    )
    inputs.extend(
        [
            """
        # ====== EXAMPLE (FOR REFERENCE ONLY) ======\n
        The following set of videos and resulting protocols should solely serve as an example and are not part of the task.\n
        Example Video 1:
        """
        ]
    )
    inputs.extend(video_example_1)
    inputs.extend(
        [
            """Example documentation 1:
    1. Describe what you can hear with timestamps:
    - No audible speech.\n
    2. Describe what you can see with timestamps:
    - 0:00-0:05 The camera pans around the back of a timsTOF mass spectrometer in a lab. A bench with various equipment is visible.
    - 0:06-0:26 A researcher takes an Eppendorf pipette, then setting volume to 1 uL.
    - 0:27-0:30 The researcher indicates that the color on the top of a pipette and of pipette tip box have to agree.
    - 0:31-0:35 the researcher attaches a pipette tip to the pipette.
    - 0:36-0:50 The researcher opens an eppendorf vial and aspirates the content.
    - 0:51-0:54 The camera moves to show the lab and the evosep instrument connected to the mass spectrometer.
    - 0:55-1:03 The researcher disconnects a tubing on the ultraSource.
    - 1:03-1:07 The researcher pipettes liquid into the fitting connecting the UltraSource of a mass spectrometer.
    - 1:07-1:08 The researcher closes conencts the tubing and fitting again.
    - 1:09-1:11 The camera focused on evosep and mass spectrometer connection.
    - 1:11-1:14 The researcher is removing the pipette tip into a wast container.
    - 1:15-1:17 The camera move far away from the evosep and the mass spectrometer.\n
    3. Describe the used equipment:
    - timsTOF mass spectrometer: Dark blue.
    - Evosep One LC system: Orange and gray.
    - Eppendorf pipette: White.
    - Pipette tips: Transparent with dark grey top.
    - Eppendorf tubes: Small clear plastic vials.
    - Various bottles and consumables on the lab bench.
    """
        ]
    )
    inputs.extend(["Example Protocol 1:"])
    inputs.extend(protocol_example_1)
    inputs.extend(["Example Video 2:"])
    inputs.extend(video_example_2)
    inputs.extend(
        [
            """Example documentation 2:
    1. Describe what you can hear with timestamps:
    - 0:00 - 0:08: "Hello, here I will show you how to reset the TIMS Control perspective. At the moment, it's not in the default state. This can be seen at this image which stands out."
    - 0:09 - 0:14: "For this, we click here at the middle and say reset perspective."
    - 0:15 - 0:16: "Yes, we want this."
    - 0:17 - 0:21: "Now we're back to the default view."\n
    2. Describe what you can see with timestamps:
    - 0:00 - 0:05: The screen displays the Bruker timsControl software interface. The window title indicates "timsTOFscp". Several panels are visible: "TIMS View" at the top left showing a 2D plot (likely ion mobility vs. m/z), a "Chromatogram View" prominently on the right side displaying two traces (green and red), and various instrument status indicators on the left (Automation, HyStar, Calibration, Vacuum).
    - 0:05 - 0:08: The narrator refers to the current layout as non-default, implying the position of the "Chromatogram View" is non-standard and an customized arrangement.
    - 0:08 - 0:11: The mouse cursor moves to the top-right of the software window, towards a set of icons. It hovers over an icon that looks like two overlapping rectangles, typically used for managing window layouts or perspectives.
    - 0:11 - 0:12: The mouse clicks on the perspective management icon. A dropdown menu appears with options including "Show View...", "Save Perspective As...", "Reset Perspective...", "Home", "Method Editor", "Maintenance", and "Monitoring".
    - 0:12 - 0:14: The mouse cursor moves down the dropdown menu and selects "Reset Perspective...".
    - 0:14 - 0:15: A confirmation dialog box titled "Reset Perspective" pops up, asking: "Do you want to reset the current 'Home' perspective to its defaults?". "Yes" and "No" buttons are presented.
    - 0:15 - 0:16: The mouse cursor clicks the "Yes" button in the confirmation dialog.
    - 0:17 - 0:21: The software interface panels dynamically rearrange. The "TIMS View" remains at the top. A "Spectrum View" now appears in the middle section, and the "Chromatogram View" is repositioned to the bottom of the main display area. This is indicative of the default layout for this software.
    - 0:21 - 0:23: The software is shown in its new, reset (default) perspective. The mouse cursor makes a brief circular movement over the "Source" and "Syringe Pump" control area in the lower part of the screen before the video ends.\n
    3. Describe the used equipment:
    - Bruker timsControl Software: The primary focus of the video. The window title "timsTOFscp - ... - timsControl" indicates this specific software, used for controlling Bruker TIMS-TOF series mass spectrometers.
    - Computer System: The software is running on a computer, evidenced by the VNC Viewer window frame and the Windows taskbar visible at the very bottom of the screen.
    - (Implied) Bruker timsTOF Mass Spectrometer: The software is designed to control a Trapped Ion Mobility Spectrometry Time-Of-Flight mass spectrometer (e.g., timsTOF Pro, timsTOF SCP, timsTOF fleX). The "TIMS View" and other parameters are specific to such instrumentation.
    """
        ]
    )
    inputs.extend(["Example Protocol:"])
    inputs.extend(protocol_example_2)
    inputs.extend(
        [
            """
        # ====== Beginn of Analysis Task ======\n
        ## Important: The analysis must be performed on the following video \n
        Video:
        """
        ]
    )
    inputs.extend(video)
    inputs.extend(
        [
            """
        Protocol:
    """
        ]
    )

    protocol_ai, usage_metadata = video_to_protocol.generate_content_from_model(
        inputs,
        model_name,
        temperature,
    )

    return protocol_ai, usage_metadata


def protocol_generation_with_thinking_step_by_step_plus_knowledge(
    video: str,
    video_example_1: str,
    protocol_example_1: str,
    video_example_2: str,
    protocol_example_2: str,
    knowledge: str,
    model_name: str,
    temperature: float,
) -> tuple[str, dict[str, Any]]:
    """Generate protocol using step-by-step thinking with knowledge.

    Parameters
    ----------
    video : str
        Input video to generate protocol for
    video_example_1 : str
        First example video
    protocol_example_1 : str
        Protocol for first example video
    video_example_2 : str
        Second example video
    protocol_example_2 : str
        Protocol for second example video
    knowledge : str
        Domain knowledge to incorporate
    model_name : str
        Name of the model to use
    temperature : float
        Temperature setting for generation

    Returns
    -------
    tuple[str, dict[str, Any]]
        A tuple containing (protocol_ai, usage_metadata)

    """
    inputs = [
        """
    You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry with extensive laboratory experience.\n
    ## ====== Background Knowledge (FOR REFERENCE ONLY) ======
    [These documents are for building your proteomics background knowledge and are not part of your task.]
    """
    ]
    inputs.extend(knowledge)
    inputs.extend(
        [
            """
    # Instruction

    You work with following input:
    - Video: An instructional video that demonstrates how a researcher carries out a laboratory procedure.\n
    Your task is to analyze the provided video and to convert it into a Nature-style protocol. The goal is a clear, concise, unambiguous protocol reproducible by someone with no prior knowledge.\n
    ## Follow this structured approach:
    * Step 1: Go through the 'Video' completely from beginning to end.
    * Step 2: Document all observations:
        - write down what you can hear with timestamps
        - write down all actions you can see with timestamps
        - note down the equipment you can identify
    * Step 3: Convert your observations into a Nature-style protocol:
        ###Protocol Writing Guidelines
        ####Format Requirements
        1. Title (format: **# Title**)
        2. Abstract (format: **## Abstract** followed by a paragraph)
        3. Materials section with Equipment and Reagents subsections
            (format: **## Materials**
                    **### Equipment**
                    - **Item 1**
                    - **Item 2**)
        4. Procedure with estimated timing
            (format: **## Procedure**
                    *Estimated timing: X minutes*
                    1. Step one
                    2. Step two)
        5. Expected Results section (format: **## Expected Results**)
        6. Figures section (format: **## Figures**)
        7. References section (format: **## References**)

        ###Key Content Adjustments
        #### Abstract
        - Focus on the core procedure, not extensive background

        #### Procedure Section
        - Steps:
            - Focus on essential actions only.
            - Be brief in your description.
            - Give every step its own number.
            - Use subheadings to group steps such as
                '### Switch timsTOF to standby
                1. In ...
                2. Verified ...
                3. In ...
                ...'
        - Language: Use direct, action-oriented language with commonly used vocabularies
        - Estimated timing: Use the video legth
    """
        ]
    )
    inputs.extend(
        [
            """
        # ====== EXAMPLE (FOR REFERENCE ONLY) ======\n
        The following set of videos and resulting protocols should solely serve as an example and are not part of the task.\n
        Example Video 1:
        """
        ]
    )
    inputs.extend(video_example_1)
    inputs.extend(
        [
            """Example documentation 1:
    1. Describe what you can hear with timestamps:
    - No audible speech.\n
    2. Describe what you can see with timestamps:
    - 0:00-0:05 The camera pans around the back of a timsTOF mass spectrometer in a lab. A bench with various equipment is visible.
    - 0:06-0:26 A researcher takes an Eppendorf pipette, then setting volume to 1 uL.
    - 0:27-0:30 The researcher indicates that the color on the top of a pipette and of pipette tip box have to agree.
    - 0:31-0:35 the researcher attaches a pipette tip to the pipette.
    - 0:36-0:50 The researcher opens an eppendorf vial and aspirates the content.
    - 0:51-0:54 The camera moves to show the lab and the evosep instrument connected to the mass spectrometer.
    - 0:55-1:03 The researcher disconnects a tubing on the ultraSource.
    - 1:03-1:07 The researcher pipettes liquid into the fitting connecting the UltraSource of a mass spectrometer.
    - 1:07-1:08 The researcher closes conencts the tubing and fitting again.
    - 1:09-1:11 The camera focused on evosep and mass spectrometer connection.
    - 1:11-1:14 The researcher is removing the pipette tip into a wast container.
    - 1:15-1:17 The camera move far away from the evosep and the mass spectrometer.\n
    3. Describe the used equipment:
    - timsTOF mass spectrometer: Dark blue.
    - Evosep One LC system: Orange and gray.
    - Eppendorf pipette: White.
    - Pipette tips: Transparent with dark grey top.
    - Eppendorf tubes: Small clear plastic vials.
    - Various bottles and consumables on the lab bench.
    """
        ]
    )
    inputs.extend(["Example Protocol 1:"])
    inputs.extend(protocol_example_1)
    inputs.extend(["Example Video 2:"])
    inputs.extend(video_example_2)
    inputs.extend(
        [
            """Example documentation 2:
    1. Describe what you can hear with timestamps:
    - 0:00 - 0:08: "Hello, here I will show you how to reset the TIMS Control perspective. At the moment, it's not in the default state. This can be seen at this image which stands out."
    - 0:09 - 0:14: "For this, we click here at the middle and say reset perspective."
    - 0:15 - 0:16: "Yes, we want this."
    - 0:17 - 0:21: "Now we're back to the default view."\n
    2. Describe what you can see with timestamps:
    - 0:00 - 0:05: The screen displays the Bruker timsControl software interface. The window title indicates "timsTOFscp". Several panels are visible: "TIMS View" at the top left showing a 2D plot (likely ion mobility vs. m/z), a "Chromatogram View" prominently on the right side displaying two traces (green and red), and various instrument status indicators on the left (Automation, HyStar, Calibration, Vacuum).
    - 0:05 - 0:08: The narrator refers to the current layout as non-default, implying the position of the "Chromatogram View" is non-standard and an customized arrangement.
    - 0:08 - 0:11: The mouse cursor moves to the top-right of the software window, towards a set of icons. It hovers over an icon that looks like two overlapping rectangles, typically used for managing window layouts or perspectives.
    - 0:11 - 0:12: The mouse clicks on the perspective management icon. A dropdown menu appears with options including "Show View...", "Save Perspective As...", "Reset Perspective...", "Home", "Method Editor", "Maintenance", and "Monitoring".
    - 0:12 - 0:14: The mouse cursor moves down the dropdown menu and selects "Reset Perspective...".
    - 0:14 - 0:15: A confirmation dialog box titled "Reset Perspective" pops up, asking: "Do you want to reset the current 'Home' perspective to its defaults?". "Yes" and "No" buttons are presented.
    - 0:15 - 0:16: The mouse cursor clicks the "Yes" button in the confirmation dialog.
    - 0:17 - 0:21: The software interface panels dynamically rearrange. The "TIMS View" remains at the top. A "Spectrum View" now appears in the middle section, and the "Chromatogram View" is repositioned to the bottom of the main display area. This is indicative of the default layout for this software.
    - 0:21 - 0:23: The software is shown in its new, reset (default) perspective. The mouse cursor makes a brief circular movement over the "Source" and "Syringe Pump" control area in the lower part of the screen before the video ends.\n
    3. Describe the used equipment:
    - Bruker timsControl Software: The primary focus of the video. The window title "timsTOFscp - ... - timsControl" indicates this specific software, used for controlling Bruker TIMS-TOF series mass spectrometers.
    - Computer System: The software is running on a computer, evidenced by the VNC Viewer window frame and the Windows taskbar visible at the very bottom of the screen.
    - (Implied) Bruker timsTOF Mass Spectrometer: The software is designed to control a Trapped Ion Mobility Spectrometry Time-Of-Flight mass spectrometer (e.g., timsTOF Pro, timsTOF SCP, timsTOF fleX). The "TIMS View" and other parameters are specific to such instrumentation.
    """
        ]
    )
    inputs.extend(["Example Protocol:"])
    inputs.extend(protocol_example_2)
    inputs.extend(
        [
            """
        # ====== Beginn of Analysis Task ======\n
        ## Important: The analysis must be performed on the following video \n
        Video:
        """
        ]
    )
    inputs.extend(video)
    inputs.extend(
        [
            """
        Protocol:
    """
        ]
    )

    protocol_ai, usage_metadata = video_to_protocol.generate_content_from_model(
        inputs,
        model_name,
        temperature,
    )

    return protocol_ai, usage_metadata


def protocol_generation_with_thinking_step_by_step_plus_knowledge_without_persona(
    video: str,
    video_example_1: str,
    protocol_example_1: str,
    video_example_2: str,
    protocol_example_2: str,
    knowledge: str,
    model_name: str,
    temperature: float,
) -> tuple[str, dict[str, Any]]:
    """Generate protocol using step-by-step thinking with knowledge but without persona.

    Parameters
    ----------
    video : str
        Input video to generate protocol for
    video_example_1 : str
        First example video
    protocol_example_1 : str
        Protocol for first example video
    video_example_2 : str
        Second example video
    protocol_example_2 : str
        Protocol for second example video
    knowledge : str
        Domain knowledge to incorporate
    model_name : str
        Name of the model to use
    temperature : float
        Temperature setting for generation

    Returns
    -------
    tuple[str, dict[str, Any]]
        A tuple containing (protocol_ai, usage_metadata)

    """
    inputs = [
        """
    ## ====== Background Knowledge (FOR REFERENCE ONLY) ======
    [These documents are for building your proteomics background knowledge and are not part of your task.]
    """
    ]
    inputs.extend(knowledge)
    inputs.extend(
        [
            """
    # Instruction

    You work with following input:
    - Video: An instructional video that demonstrates how a researcher carries out a laboratory procedure.\n
    Your task is to analyze the provided video and to convert it into a Nature-style protocol. The goal is a clear, concise, unambiguous protocol reproducible by someone with no prior knowledge.\n
    ## Follow this structured approach:
    * Step 1: Go through the 'Video' completely from beginning to end.
    * Step 2: Document all observations:
        - write down what you can hear with timestamps
        - write down all actions you can see with timestamps
        - note down the equipment you can identify
    * Step 3: Convert your observations into a Nature-style protocol:
        ###Protocol Writing Guidelines
        ####Format Requirements
        1. Title (format: **# Title**)
        2. Abstract (format: **## Abstract** followed by a paragraph)
        3. Materials section with Equipment and Reagents subsections
            (format: **## Materials**
                    **### Equipment**
                    - **Item 1**
                    - **Item 2**)
        4. Procedure with estimated timing
            (format: **## Procedure**
                    *Estimated timing: X minutes*
                    1. Step one
                    2. Step two)
        5. Expected Results section (format: **## Expected Results**)
        6. Figures section (format: **## Figures**)
        7. References section (format: **## References**)

        ###Key Content Adjustments
        #### Abstract
        - Focus on the core procedure, not extensive background

        #### Procedure Section
        - Steps:
            - Focus on essential actions only.
            - Be brief in your description.
            - Give every step its own number.
            - Use subheadings to group steps such as
                '### Switch timsTOF to standby
                1. In ...
                2. Verified ...
                3. In ...
                ...'
        - Language: Use direct, action-oriented language with commonly used vocabularies
        - Estimated timing: Use the video legth
    """
        ]
    )
    inputs.extend(
        [
            """
        # ====== EXAMPLE (FOR REFERENCE ONLY) ======\n
        The following set of videos and resulting protocols should solely serve as an example and are not part of the task.\n
        Example Video 1:
        """
        ]
    )
    inputs.extend(video_example_1)
    inputs.extend(
        [
            """Example documentation 1:
    1. Describe what you can hear with timestamps:
    - No audible speech.\n
    2. Describe what you can see with timestamps:
    - 0:00-0:05 The camera pans around the back of a timsTOF mass spectrometer in a lab. A bench with various equipment is visible.
    - 0:06-0:26 A researcher takes an Eppendorf pipette, then setting volume to 1 uL.
    - 0:27-0:30 The researcher indicates that the color on the top of a pipette and of pipette tip box have to agree.
    - 0:31-0:35 the researcher attaches a pipette tip to the pipette
    - 0:36-0:50 The researcher opens an eppendorf vial and aspirates the content.
    - 0:51-0:54 The camera moves to show the lab and the evosep instrument connected to the mass spectrometer.
    - 0:55-1:03 The researcher disconnects a tubing on the ultraSource.
    - 1:03-1:07 The researcher pipettes liquid into the fitting connecting the UltraSource of a mass spectrometer.
    - 1:07-1:08 The researcher closes conencts the tubing and fitting again.
    - 1:09-1:11 The camera focused on evosep and mass spectrometer connection.
    - 1:11-1:14 The researcher is removing the pipette tip into a wast container.
    - 1:15-1:17 The camera move far away from the evosep and the mass spectrometer.\n
    3. Describe the used equipment:
    - timsTOF mass spectrometer: Dark blue.
    - Evosep One LC system: Orange and gray.
    - Eppendorf pipette: White.
    - Pipette tips: Transparent with dark grey top.
    - Eppendorf tubes: Small clear plastic vials.
    - Various bottles and consumables on the lab bench.
    """
        ]
    )
    inputs.extend(["Example Protocol 1:"])
    inputs.extend(protocol_example_1)
    inputs.extend(["Example Video 2:"])
    inputs.extend(video_example_2)
    inputs.extend(
        [
            """Example documentation 2:
    1. Describe what you can hear with timestamps:
    - 0:00 - 0:08: "Hello, here I will show you how to reset the TIMS Control perspective. At the moment, it's not in the default state. This can be seen at this image which stands out."
    - 0:09 - 0:14: "For this, we click here at the middle and say reset perspective."
    - 0:15 - 0:16: "Yes, we want this."
    - 0:17 - 0:21: "Now we're back to the default view."\n
    2. Describe what you can see with timestamps:
    - 0:00 - 0:05: The screen displays the Bruker timsControl software interface. The window title indicates "timsTOFscp". Several panels are visible: "TIMS View" at the top left showing a 2D plot (likely ion mobility vs. m/z), a "Chromatogram View" prominently on the right side displaying two traces (green and red), and various instrument status indicators on the left (Automation, HyStar, Calibration, Vacuum).
    - 0:05 - 0:08: The narrator refers to the current layout as non-default, implying the position of the "Chromatogram View" is non-standard and an customized arrangement.
    - 0:08 - 0:11: The mouse cursor moves to the top-right of the software window, towards a set of icons. It hovers over an icon that looks like two overlapping rectangles, typically used for managing window layouts or perspectives.
    - 0:11 - 0:12: The mouse clicks on the perspective management icon. A dropdown menu appears with options including "Show View...", "Save Perspective As...", "Reset Perspective...", "Home", "Method Editor", "Maintenance", and "Monitoring".
    - 0:12 - 0:14: The mouse cursor moves down the dropdown menu and selects "Reset Perspective...".
    - 0:14 - 0:15: A confirmation dialog box titled "Reset Perspective" pops up, asking: "Do you want to reset the current 'Home' perspective to its defaults?". "Yes" and "No" buttons are presented.
    - 0:15 - 0:16: The mouse cursor clicks the "Yes" button in the confirmation dialog.
    - 0:17 - 0:21: The software interface panels dynamically rearrange. The "TIMS View" remains at the top. A "Spectrum View" now appears in the middle section, and the "Chromatogram View" is repositioned to the bottom of the main display area. This is indicative of the default layout for this software.
    - 0:21 - 0:23: The software is shown in its new, reset (default) perspective. The mouse cursor makes a brief circular movement over the "Source" and "Syringe Pump" control area in the lower part of the screen before the video ends.\n
    3. Describe the used equipment:
    - Bruker timsControl Software: The primary focus of the video. The window title "timsTOFscp - ... - timsControl" indicates this specific software, used for controlling Bruker TIMS-TOF series mass spectrometers.
    - Computer System: The software is running on a computer, evidenced by the VNC Viewer window frame and the Windows taskbar visible at the very bottom of the screen.
    - (Implied) Bruker timsTOF Mass Spectrometer: The software is designed to control a Trapped Ion Mobility Spectrometry Time-Of-Flight mass spectrometer (e.g., timsTOF Pro, timsTOF SCP, timsTOF fleX). The "TIMS View" and other parameters are specific to such instrumentation.
    """
        ]
    )
    inputs.extend(["Example Protocol:"])
    inputs.extend(protocol_example_2)
    inputs.extend(
        [
            """
        # ====== Beginn of Analysis Task ======\n
        ## Important: The analysis must be performed on the following video \n
        Video:
        """
        ]
    )
    inputs.extend(video)
    inputs.extend(
        [
            """
        Protocol:
    """
        ]
    )

    protocol_ai, usage_metadata = video_to_protocol.generate_content_from_model(
        inputs,
        model_name,
        temperature,
    )

    return protocol_ai, usage_metadata

In [ ]:
csv_path = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/benchmark_dataset.csv"
protocol_videos_base = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/protocols"
markdown_base = "/Users/patriciaskowronek/Documents/proteomics_specialist/data"
prefix = "generate_protocol_video"

all_model_inputs = process_benchmark_dataset(
    csv_path, protocol_videos_base, markdown_base, bucket, prefix
)

In [ ]:
prefix = "fewShotExamples"

video_path_example_1 = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/Refilling_tuneMix_at_timsTOFUltra.mp4"
protocol_path_example_1 = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/Refilling_tuneMix_at_timsTOFUltra.md"
video_path_example_2 = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/reset_timsControl.mov"
protocol_path_example_2 = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/reset_timsControl.md"

dict_example_1 = prepare_all_inputs(
    video_path_example_1,
    protocol_path_example_1,
    bucket,
    prefix,
)

dict_example_2 = prepare_all_inputs(
    video_path_example_2,
    protocol_path_example_2,
    bucket,
    prefix,
)

In [ ]:
folder_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/knowledge_base"
subfolder_in_bucket = "knowledge"
unspecific_knowledge = prepare_knowledge(folder_path, bucket, subfolder_in_bucket)

folder_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/knowledge_base_selected"
specific_knowledge = prepare_knowledge(folder_path, bucket, subfolder_in_bucket)

In [ ]:
def safe_json_dump(data: dict[str, Any], filename: str | Path) -> None:
    """Handles non-serializable objects and converts items to strings.

    Parameters
    ----------
    data : dict[str, Any]
        Dictionary containing the data to be serialized
    filename : str | Path
        Path to the output JSON file

    """
    path = Path(filename)
    temp_file = path.with_suffix(f"{path.suffix}.tmp")
    with temp_file.open("w") as f:
        json.dump(serialize(data), f)
    temp_file.replace(path)


def serialize(obj: object) -> dict | list | str | int | float | bool | None:
    """Recursively serialize objects to JSON-compatible types."""
    if isinstance(obj, dict):
        return {k: serialize(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [serialize(item) for item in obj]
    if isinstance(obj, (int, float, str, bool)) or obj is None:
        return obj
    return str(obj)


def raise_evaluation_error() -> NoReturn:
    """Raise an error when evaluation contains separator line."""
    raise ValueError("Evaluation contains separator line")

In [ ]:
model_name = "gemini-2.5-pro-preview-03-25"
temperature = 0.9

function_configs = [
    {
        "name": "without_added_knowledge",
        "function": protocol_generation_without_added_knowledge,
        "knowledge": unspecific_knowledge,
    },
    {
        "name": "with_examples",
        "function": protocol_generation_with_examples,
        "knowledge": unspecific_knowledge,
    },
    {
        "name": "with_thinking_step_by_step",
        "function": protocol_generation_with_thinking_step_by_step,
        "knowledge": unspecific_knowledge,
    },
    {
        "name": "with_thinking_step_by_step_plus_unspecific_knowledge",
        "function": protocol_generation_with_thinking_step_by_step_plus_knowledge,
        "knowledge": unspecific_knowledge,
    },
    {
        "name": "with_thinking_step_by_step_plus_specific_knowledge",
        "function": protocol_generation_with_thinking_step_by_step_plus_knowledge,
        "knowledge": specific_knowledge,
    },
    {
        "name": "with_thinking_step_by_step_plus_specific_knowledge_without_persona",
        "function": protocol_generation_with_thinking_step_by_step_plus_knowledge_without_persona,
        "knowledge": specific_knowledge,
    },
]

In [ ]:
import json
import logging
import time
from pathlib import Path

# Constants for retry logic
WAIT_TIME_BETWEEN_ITEMS = 10  # seconds
RETRY_WAIT_TIME = 120  # seconds
MAX_RETRIES = 3


CHECKPOINT_FILE = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/results_protocol_generation/protocol_results_checkpoint.json"

# Load checkpoint
results_collection = {}
last_processed_key = None
last_processed_config = None

try:
    checkpoint_path = Path(CHECKPOINT_FILE)
    if checkpoint_path.exists():
        with checkpoint_path.open() as f:
            data = json.load(f)
            results_collection = data.get("results", {})
            last_processed_key = data.get("last_key", None)
            last_processed_config = data.get("last_config", None)
        print(
            f"Loaded checkpoint. Last processed key: {last_processed_key}, config: {last_processed_config}"
        )
except (json.JSONDecodeError, PermissionError, FileNotFoundError) as e:
    print(f"Error loading checkpoint: {e}")

items_list = list(all_model_inputs.items())

# Determine starting point
start_index = 0
start_config_index = 0

if last_processed_key:
    start_index = next(
        (i for i, (k, _) in enumerate(items_list) if k == last_processed_key), 0
    )
    if last_processed_config:
        start_config_index = (
            next(
                (
                    i
                    for i, config in enumerate(function_configs)
                    if config["name"] == last_processed_config
                ),
                0,
            )
            + 1
        )
        if start_config_index >= len(function_configs):
            start_index += 1
            start_config_index = 0

for i in range(start_index, len(items_list)):
    key, value = items_list[i]
    print(f"\nProcessing item: {key}")

    # Determine config starting point for this item
    config_start = start_config_index if i == start_index else 0

    for config_idx in range(config_start, len(function_configs)):
        config = function_configs[config_idx]

        for attempt in range(MAX_RETRIES):
            try:
                print(f"\n--- {config['name']} (attempt {attempt + 1}) ---")

                start_time = time.time()
                (
                    evaluation,
                    ratings_dict,
                    usage_metadata_protocol,
                    usage_metadata_eval,
                ) = generate_protocol_and_evaluate(
                    value["protocol_video_input"],
                    value["protocol_input"],
                    dict_example_1["protocol_video_input"],
                    dict_example_1["protocol_input"],
                    dict_example_2["protocol_video_input"],
                    dict_example_2["protocol_input"],
                    config["knowledge"],
                    model_name,
                    temperature,
                    selected_function=config["function"],
                )
                end_time = time.time()
                processing_time = end_time - start_time

                print(f"Processing time: {processing_time:.2f} seconds")
                display(Markdown(evaluation))
                display(ratings_dict)

                if "-" * 500 in evaluation:
                    raise_evaluation_error()

                # Store results
                if key not in results_collection:
                    results_collection[key] = {}

                results_collection[key][config["name"]] = {
                    "inputs": {
                        "experiment_name": key,
                        "config_name": config["name"],
                        "model_name": model_name,
                        "temperature": temperature,
                    },
                    "outputs": {
                        "evaluation": evaluation,
                        "ratings_dict": ratings_dict,
                        "usage_metadata_protocol": usage_metadata_protocol,
                        "usage_metadata_eval": usage_metadata_eval,
                        "processing_time": processing_time,
                    },
                }

                # Save checkpoint after each successful config
                safe_json_dump(
                    {
                        "last_key": key,
                        "last_config": config["name"],
                        "results": results_collection,
                    },
                    CHECKPOINT_FILE,
                )

                print(
                    f"Waiting {WAIT_TIME_BETWEEN_ITEMS} seconds before next config..."
                )
                time.sleep(WAIT_TIME_BETWEEN_ITEMS)
                break  # Success, exit retry loop

            except Exception:
                logging.exception(
                    f"Unexpected error processing {key} with {config['name']}"
                )
                if attempt < MAX_RETRIES - 1:
                    logging.info(f"Waiting {RETRY_WAIT_TIME} seconds before retry...")
                    time.sleep(RETRY_WAIT_TIME)
                else:
                    logging.exception(
                        f"Max retries reached for {key} with {config['name']}, moving to next config"
                    )
                    # Save checkpoint even on failure
                    safe_json_dump(
                        {
                            "last_key": key,
                            "last_config": config["name"],
                            "results": results_collection,
                        },
                        CHECKPOINT_FILE,
                    )

# Save final results
try:
    timestamp = time.time()
    results_path = Path(CHECKPOINT_FILE).parent
    final_results_file = results_path / f"final_protocol_results_{timestamp}.json"
    safe_json_dump(results_collection, final_results_file)
    print("All processing complete. Final results saved.")
except (PermissionError, OSError) as e:
    print(f"Error saving final results: {e}")

In [ ]:
# Logic to repeat analysis of single videos

CHECKPOINT_FILE = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/results_protocol_generation/protocol_results_checkpoint9.json"

# Define specific items to process with their configurations
ITEMS_TO_PROCESS = {
    "UltraSourceToESIsource_protocolCorrect": [
        "with_thinking_step_by_step_plus_unspecific_knowledge"
    ],
    "DisconnectColumn_protocolCorrect": ["without_added_knowledge"],
    "Evotip_protocolCorrect": ["with_thinking_step_by_step"],
    "TimsCalibration_protocolCorrect": ["with_examples"],
    "TimsCalibration_protocolCorrect": [
        "with_thinking_step_by_step_plus_specific_knowledge_without_persona"
    ],
    "QueueSamples_protocolCorrect": [
        "with_thinking_step_by_step_plus_specific_knowledge"
    ],
    "QueueSamples_protocolCorrect": [
        "with_thinking_step_by_step_plus_specific_knowledge_without_persona"
    ],
}

# Load checkpoint
results_collection = {}
last_processed_key = None
last_processed_config = None

try:
    checkpoint_path = Path(CHECKPOINT_FILE)
    if checkpoint_path.exists():
        with checkpoint_path.open() as f:
            data = json.load(f)
            results_collection = data.get("results", {})
            last_processed_key = data.get("last_key", None)
            last_processed_config = data.get("last_config", None)
        print(
            f"Loaded checkpoint. Last processed key: {last_processed_key}, config: {last_processed_config}"
        )
except (json.JSONDecodeError, PermissionError, FileNotFoundError) as e:
    print(f"Error loading checkpoint: {e}")

# Create a list of (key, config_name) tuples to process
items_to_process_list = []
for key, config_names in ITEMS_TO_PROCESS.items():
    items_to_process_list.extend((key, config_name) for config_name in config_names)

# Determine starting point based on checkpoint
start_index = 0
if last_processed_key and last_processed_config:
    checkpoint_tuple = (last_processed_key, last_processed_config)
    try:
        start_index = items_to_process_list.index(checkpoint_tuple) + 1
    except ValueError:
        # If checkpoint not found in our list, start from beginning
        start_index = 0

print(f"Starting processing from index {start_index}")
print(f"Total items to process: {len(items_to_process_list)}")

for i in range(start_index, len(items_to_process_list)):
    key, config_name = items_to_process_list[i]

    # Check if key exists in all_model_inputs
    if key not in all_model_inputs:
        print(f"Warning: Key '{key}' not found in all_model_inputs, skipping...")
        continue

    # Find the config in function_configs
    config = None
    for cfg in function_configs:
        if cfg["name"] == config_name:
            config = cfg
            break

    if config is None:
        print(
            f"Warning: Config '{config_name}' not found in function_configs, skipping..."
        )
        continue

    value = all_model_inputs[key]
    print(
        f"\nProcessing item: {key} with config: {config_name} ({i + 1}/{len(items_to_process_list)})"
    )

    for attempt in range(MAX_RETRIES):
        try:
            print(f"\n--- {config['name']} (attempt {attempt + 1}) ---")

            start_time = time.time()
            evaluation, ratings_dict, usage_metadata_protocol, usage_metadata_eval = (
                generate_protocol_and_evaluate(
                    value["protocol_video_input"],
                    value["protocol_input"],
                    dict_example_1["protocol_video_input"],
                    dict_example_1["protocol_input"],
                    dict_example_2["protocol_video_input"],
                    dict_example_2["protocol_input"],
                    config["knowledge"],
                    model_name,
                    temperature,
                    selected_function=config["function"],
                )
            )
            end_time = time.time()
            processing_time = end_time - start_time

            print(f"Processing time: {processing_time:.2f} seconds")
            display(Markdown(evaluation))
            display(ratings_dict)

            if "-" * 500 in evaluation:
                raise_evaluation_error()

            # Store results
            if key not in results_collection:
                results_collection[key] = {}

            results_collection[key][config["name"]] = {
                "inputs": {
                    "experiment_name": key,
                    "config_name": config["name"],
                    "model_name": model_name,
                    "temperature": temperature,
                },
                "outputs": {
                    "evaluation": evaluation,
                    "ratings_dict": ratings_dict,
                    "usage_metadata_protocol": usage_metadata_protocol,
                    "usage_metadata_eval": usage_metadata_eval,
                    "processing_time": processing_time,
                },
            }

            # Save checkpoint after each successful config
            safe_json_dump(
                {
                    "last_key": key,
                    "last_config": config["name"],
                    "results": results_collection,
                },
                CHECKPOINT_FILE,
            )

            print(f"Successfully processed {key} with {config_name}")

            # Only wait if there are more items to process
            if i < len(items_to_process_list) - 1:
                print(f"Waiting {WAIT_TIME_BETWEEN_ITEMS} seconds before next item...")
                time.sleep(WAIT_TIME_BETWEEN_ITEMS)

            break  # Success, exit retry loop

        except Exception:
            logging.exception(
                f"Unexpected error processing {key} with {config['name']}"
            )
            if attempt < MAX_RETRIES - 1:
                logging.info(f"Waiting {RETRY_WAIT_TIME} seconds before retry...")
                time.sleep(RETRY_WAIT_TIME)
            else:
                logging.exception(
                    f"Max retries reached for {key} with {config['name']}, moving to next item"
                )
                # Save checkpoint even on failure
                safe_json_dump(
                    {
                        "last_key": key,
                        "last_config": config["name"],
                        "results": results_collection,
                    },
                    CHECKPOINT_FILE,
                )

# Save final results
try:
    timestamp = time.time()
    results_path = Path(CHECKPOINT_FILE).parent
    final_results_file = results_path / f"final_protocol_results_{timestamp}.json"
    safe_json_dump(results_collection, final_results_file)
    print("All processing complete. Final results saved.")
    print(
        f"Processed {len([k for k in results_collection if k in ITEMS_TO_PROCESS])} items total"
    )
except (PermissionError, OSError) as e:
    print(f"Error saving final results: {e}")

In [ ]:
content = """
Step 2: AI-Generated Protocol (Verbatim)\n\n# Connecting an IonOpticks Column to a Bruker timsTOF SCP Mass Spectrometer and Initiating LC-MS Operation\n\n## Abstract\nThis protocol details the procedure for connecting an IonOpticks analytical column, via its sample line, to the UltraSource of a Bruker timsTOF SCP mass spectrometer. It includes steps for verifying system readiness in timsControl and HyStar software, making the fluidic connection, ensuring proper column heating and grounding, and initiating system operation to observe a signal. This procedure is essential for preparing the LC-MS system for data acquisition.\n\n## Materials\n\n### Equipment\n- **Bruker timsTOF SCP Mass Spectrometer:** Equipped with an UltraSource and integrated column oven.\n- **Evosep One LC System:** (or compatible nanoLC system)\n- **IonOpticks Column:** Pre-installed in the column oven assembly (e.g., Aurora Series).\n- **Sample Line:** Fused silica capillary with appropriate nano-fitting (e.g., PEEK fitting) for connection to the IonOpticks column inlet.\n- **Metal Adapter (optional, for tightening):** To aid in gripping the sample line fitting if necessary (removed after connection).\n- **Flat-Jawed Pliers:** For counter-holding the column inlet fitting during connection (e.g., Knipex Pliers Wrench).\n- **Computer Workstation:** Running Bruker timsControl and Bruker HyStar software.\n- **Personal Protective Equipment:** Nitrile gloves.\n\n### Reagents\n- Solvents for LC system (appropriate for column flushing and operation, e.g., mobile phases A and B).\n\n## Procedure\n*Estimated timing: Approximately 3 minutes*\n\n### 1. Verify Mass Spectrometer Status in timsControl\n1.  Navigate to the Bruker timsControl software.\n2.  Check the "Automation" status panel. If the system is in "Operating" mode, click on the "HyStar" button (or equivalent control for your LC) to switch the mass spectrometer to "Standby by" mode.\n*   **CRITICAL STEP** For IonOpticks columns, ensure the MS is not in "Operate" mode for extended periods without flow, and conversely, ensure it is in "Standby" if flow is active but no acquisition is running, to prevent damage.\n\n### 2. Prepare and Connect the Sample Line to the IonOpticks Column\n1.  Confirm that the IonOpticks column is already installed in the column oven, and the oven is mounted on the UltraSource.\n2.  Take the sample line originating from the LC system.\n3.  If using a metal adapter for easier tightening, push it onto the PEEK fitting of the sample line.\n4.  Inspect the open end of the sample line capillary. Ensure it is clean and patent. If necessary, carefully trim the end.\n5.  Carefully hold the inlet fitting of the installed IonOpticks column using flat-jawed pliers to provide counter-torque.\n6.  Insert the sample line fitting (with or without the metal adapter) into the column's inlet fitting.\n7.  Hand-tighten the sample line fitting into the column connector until finger-tight. **CAUTION:** Do not overtighten, as this can damage the fittings or the column.\n8.  If a metal adapter was used on the sample line fitting, remove it now.\n\n### 3. Verify Column Oven and Grounding\n1.  Ensure the column oven assembly is positioned correctly, typically as close as possible to the ion source inlet. Adjust using the positioning screw if necessary.\n2.  Verify that a grounding screw is making contact with the column assembly or its connector to ensure proper grounding. Multiple grounding points may be available; ensure at least one is correctly engaged.\n3.  Close the column oven lid. A click should be audible.\n4.  Observe the LED indicator lights on the column oven. Three blinking green lights indicate the oven is heating to its setpoint (e.g., 50 °C). The lights will become solid once the temperature is reached.\n\n### 4. Initiate System Operation and Verify Signal\n1.  Return to the Bruker timsControl software.\n2.  Click on the "HyStar" button (or equivalent LC control) in the "Automation" panel to switch the mass spectrometer to "Operating" mode.\n3.  Navigate to the Bruker HyStar software (or your LC control software).\n4.  Verify that "Idle Flow" is active. If not, initiate idle flow (e.g., by right-clicking and selecting an "Idle Flow run" or similar command).\n5.  Once the LC system indicates flow and the MS is in operate mode, observe the timsControl software for a stable spray and detection of ions (e.g., background ions from the mobile phase).\n\n## Expected Results\n- The sample line is securely connected to the IonOpticks column without leaks.\n- The column oven is closed and heating to the set temperature (e.g., 50 °C), indicated by the LEDs.\n- The column is properly grounded.\n- The timsTOF SCP mass spectrometer is in "Operating" mode.\n- The LC system is delivering flow (idle flow).\n- A stable ion signal is observable in the timsControl software, indicating the system is ready for sample analysis.\n\n## Figures\n### Figure 1: Software Status Check in timsControl.\n[Placeholder for a screenshot showing the timsControl interface with the "Automation" panel, highlighting the switch from "Operating" to "Standby by" (0:16-0:24).]\n\n### Figure 2: Connecting the Sample Line to the IonOpticks Column.\n[Placeholder for an image depicting the hand-tightening of the sample line fitting into the column inlet fitting, with pliers used for counter-holding (1:06-1:16).]\n\n### Figure 3: Column Oven and Grounding Verification.\n[Placeholder for an image showing the closed column oven with blinking/solid green LED temperature indicators and a finger pointing to a grounding screw (2:00-2:10).]\n\n### Figure 4: System Operating with Signal in timsControl.\n[Placeholder for a screenshot of the timsControl interface in "Operating" mode, displaying chromatogram and mass spectrum views indicating a live signal (2:38-2:41).]\n\n## References\n1.  Bruker timsTOF Series User Manuals.\n2.  Evosep One User Manual (or relevant LC system manual).\n3.  IonOpticks Column Handling and Installation Guide.\n\n###"""

display(Markdown(content))